In [23]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [24]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [25]:
text[:200]

"Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverything was in confusion in the Oblonskys' house. The wife had\ndiscovered that the husband was carrying on"

In [26]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [27]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    shape_0 = one_hot.shape[0]
    flattened_arr = arr.flatten()
    one_hot[np.arange(shape_0), flattened_arr] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [28]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [29]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr)//(batch_size*seq_length)
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batches*batch_size*seq_length]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [30]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [31]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 9 63 54  1 67 10 40 78 31 56]
 [55 53 59 78 67 63 54 67 78 54]
 [10 59 45 78 53 40 78 54 78 64]
 [55 78 67 63 10 78 71 63 76 10]
 [78 55 54 47 78 63 10 40 78 67]
 [71 68 55 55 76 53 59 78 54 59]
 [78 46 59 59 54 78 63 54 45 78]
 [23  0 22 53 59 55 16  4 42 78]]

y
 [[63 54  1 67 10 40 78 31 56 56]
 [53 59 78 67 63 54 67 78 54 67]
 [59 45 78 53 40 78 54 78 64 53]
 [78 67 63 10 78 71 63 76 10 64]
 [55 54 47 78 63 10 40 78 67 10]
 [68 55 55 76 53 59 78 54 59 45]
 [46 59 59 54 78 63 54 45 78 55]
 [ 0 22 53 59 55 16  4 42 78 77]]


In [32]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [33]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        input_length = len(self.chars)
        self.lstm = nn.LSTM(input_length, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, input_length)
        
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [34]:
def train(net, data, epochs=2, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [35]:
# define and print the net
n_hidden=20
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 20, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=20, out_features=83, bias=True)
)


In [36]:
batch_size = 10
seq_length = 50
n_epochs = 2 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/2... Step: 10... Loss: 4.4009... Val Loss: 4.3972
Epoch: 1/2... Step: 20... Loss: 4.3418... Val Loss: 4.3232
Epoch: 1/2... Step: 30... Loss: 4.2021... Val Loss: 4.1877
Epoch: 1/2... Step: 40... Loss: 3.9893... Val Loss: 3.9106
Epoch: 1/2... Step: 50... Loss: 3.6939... Val Loss: 3.5792
Epoch: 1/2... Step: 60... Loss: 3.5191... Val Loss: 3.3805
Epoch: 1/2... Step: 70... Loss: 3.3855... Val Loss: 3.2753
Epoch: 1/2... Step: 80... Loss: 3.3949... Val Loss: 3.2181
Epoch: 1/2... Step: 90... Loss: 3.3095... Val Loss: 3.1863
Epoch: 1/2... Step: 100... Loss: 3.1949... Val Loss: 3.1705
Epoch: 1/2... Step: 110... Loss: 3.2228... Val Loss: 3.1613
Epoch: 1/2... Step: 120... Loss: 3.2536... Val Loss: 3.1551
Epoch: 1/2... Step: 130... Loss: 3.1891... Val Loss: 3.1500
Epoch: 1/2... Step: 140... Loss: 3.2450... Val Loss: 3.1443
Epoch: 1/2... Step: 150... Loss: 3.2719... Val Loss: 3.1401
Epoch: 1/2... Step: 160... Loss: 3.2317... Val Loss: 3.1379
Epoch: 1/2... Step: 170... Loss: 3.1283... Val Lo

KeyboardInterrupt: 

In [39]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [40]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [41]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annaeaee aa ht otee ot o taaet aaao h otiaae ana h n h to h hateetat n oni oa n h a oht t he h n ohatinaoe a heant oh nat taa he h a o h ne oniaee on on h tooaa oa hin on tte ahee nt oaoao at ttaeteaaoo an h anite on ate hiatia te neeae n hte an a aaaooeet ha oa h t ohe naeiae nt aaeeit on oh n no ahaoo hat het taao he n heeiant anie hine te htaeeanea te hateeeet ht noo n hi h teihe ohtit t toot neihtt o t anit ate ah aht atein at oheat ottei nte t heiae n nea tt at o ha n aa to h ohi ta htt o t haei ata no t teeineihi haee taoee ta ttiaaa heiha n hia h heih h h haetin aa ht hie tatee tat tee an naaa taeiae te tt ttiao h tteetao ht tee ne he on h hteet aaet hee an atia o n toeint tt nooae neitaatetaate h ahi anee t ateeinaoae anaee t nttit ha hateit tet t ne tt nea n nt o het aat ohe tnaoete n anit h t on oaae a a te a a ateee nt ot oa anao an tett h he a atte teao nao naataoaee t haa n naa haata h o t ne tne ht h note t an neat ot ha ha otao ttateet n htiea h n o ae a oni tee atee an 